In [1]:
import csv
import numpy
from IPython.display import display, Markdown
X = []
y = []
with open('abalone.csv') as myfile:
    data = csv.reader(myfile, delimiter=',')
    #Στον πίνακα Χ δεν περιλαμβάνουμε την μεταβλητή sex επειδή δεν είναι αριθμητική
    #Στον πίνακα y παίρνουμε την μεταβλητη Rings που είναι το target μας
    for dataline in data:
        y.append(dataline[8:])
        X.append(dataline[1:8])
        
X = numpy.array(X)
y = numpy.array(y)

X = X.astype(float)
y = y.astype(float)

print("Data shapes: X:{}, y:{}".format(numpy.shape(X), numpy.shape(y)) )
shape =  numpy.shape(X)
print("Αντικείμενα:",shape[0])
print("Μεταβλητές:",shape[1])
print("Εύρος τιμών y :R=",numpy.max(y)-numpy.min(y))

print("Μέσος μεταβλητών:",X.mean(axis=0))
print("Τυπική αποκλίση μεταβλητών:",X.std(axis=0))  
print (X.var(axis=0))
print("Μέσος στόχου:",numpy.mean(y))
print("Τυπική απόκλιση στόχου:",numpy.std(y))



Data shapes: X:(4177, 7), y:(4177, 1)
Αντικείμενα: 4177
Μεταβλητές: 7
Εύρος τιμών y :R= 28.0
Μέσος μεταβλητών: [0.5239921  0.40788125 0.1395164  0.82874216 0.35936749 0.18059361
 0.23883086]
Τυπική αποκλίση μεταβλητών: [0.12007854 0.09922799 0.04182205 0.49033031 0.22193638 0.10960113
 0.13918601]
[0.01441885 0.00984619 0.00174908 0.24042382 0.04925576 0.01201241
 0.01937274]
Μέσος στόχου: 9.933684462532918
Τυπική απόκλιση στόχου: 3.2237830658212117


**Επειδή οι τιμές των δεδομένων έχουν μέσο και τυπική αποκλίση κοντα στο 0 και 1 μάλλον η κανονικοποιήση δεν είναι απαραίτητη αλλά θα κάνουμε για να ελέγξουμε**

In [2]:
from sklearn.preprocessing import StandardScaler



#Διαχωρισμος δεδομένων
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=0)


#Κανονικοποιηση πινάκων
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)





In [3]:
def success_rate(y_test_pred,y_test,N):
    succeses=0
   
    for i in range(len(y_test_pred)):
        
        if numpy.less_equal(numpy.abs(y_test_pred[i]-y_test[i]),N):
            succeses=succeses+1
    return succeses/len(y_test_pred)*100
        
        
    
    
    

In [4]:
#Παλινδρόμηση για κανονικοποιημενα δεδομενα
from sklearn.metrics import explained_variance_score
from sklearn import  linear_model


regr = linear_model.LinearRegression()
regr.fit(X_train_scaled, y_train)
y_test_pred_scaled = regr.predict(X_test_scaled)


display(Markdown(
    "### Αποτελέσματα παλινδρόμησης για κανονικοποιημένα δεδομένα "+ ":\n" +
    "**Test set explained variance**: {:.2f}".format(explained_variance_score(y_test, y_test_pred_scaled))+"<br>"+
    "**Success ratio**: {:.2f}".format(success_rate(y_test_pred_scaled,y_test,1))+"%"+"<br>"+
    "**Coefficients from all data(W)**:" + str(regr.coef_)
))





regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_test_pred = regr.predict(X_test)



display(Markdown(
    "### Αποτελέσματα παλινδρόμησης για αρχικά δεδομένα "+ ":\n" +
    "**Test set explained variance**: {:.2f}".format(explained_variance_score(y_test, y_test_pred))+"<br>"+
    "**Success ratio**: {:.2f}".format(success_rate(y_test_pred,y_test,1))+"%"+"<br>"+
    "**Coefficients from all data(W)**:" + str(regr.coef_)
))






### Αποτελέσματα παλινδρόμησης για κανονικοποιημένα δεδομένα :
**Test set explained variance**: 0.54<br>**Success ratio**: 42.82%<br>**Coefficients from all data(W)**:[[-0.11023424  1.27841734  0.43275322  4.70895912 -4.54132045 -1.15485229
   1.15646108]]

### Αποτελέσματα παλινδρόμησης για αρχικά δεδομένα :
**Test set explained variance**: 0.54<br>**Success ratio**: 42.82%<br>**Coefficients from all data(W)**:[[ -0.92025855  12.91880135  10.27163765   9.59385694 -20.42655793
  -10.53794044   8.29607609]]

In [5]:
#Ridge regression με grid search
from sklearn.model_selection import GridSearchCV
params = { 
           'alpha': numpy.logspace(-6, 4, 20)
         }

regr  = linear_model.Ridge() 
grid = GridSearchCV(estimator=regr, cv = 10, param_grid=params) 
grid.fit(X_train_scaled, y_train) 
y_train_pred_scaled = grid.predict(X_train_scaled)
y_test_pred_scaled = grid.predict(X_test_scaled)



display(Markdown(
    "### Αποτελέσματα ridge regression με grid search για κανονικοποιημένα δεδομένα "+ ":\n" +
    "**Cross-val explained variance**: {:.2f}".format(explained_variance_score(y_train, y_train_pred_scaled)) + "<br>" +
    "**Test set explained variance**: {:.2f}".format(explained_variance_score(y_test, y_test_pred_scaled))+"<br>"+
    "**Success ratio**: {:.2f}".format(success_rate(y_test_pred_scaled,y_test,1))+"%"+"<br>"+
    "**Coefficients from all data(W)**:" + str(grid.best_estimator_.coef_)
))


regr4 = linear_model.Ridge() 
grid = GridSearchCV(estimator=regr, cv = 10, param_grid=params) 
grid.fit(X_train, y_train) 
y_train_pred = grid.predict(X_train)
y_test_pred = grid.predict(X_test)


display(Markdown(
    "### Αποτελέσματα ridge regression με grid search για αρχικά δεδομένα "+ ":\n" +
    "**Cross-val explained variance**: {:.2f}".format(explained_variance_score(y_train, y_train_pred)) + "<br>" +
    "**Test set explained variance**: {:.2f}".format(explained_variance_score(y_test, y_test_pred))+"<br>"+
    "**Success ratio**: {:.2f}".format(success_rate(y_test_pred,y_test,1))+"%"+"<br>"+
    "**Coefficients from all data(W)**:" + str(grid.best_estimator_.coef_)
))


### Αποτελέσματα ridge regression με grid search για κανονικοποιημένα δεδομένα :
**Cross-val explained variance**: 0.53<br>**Test set explained variance**: 0.54<br>**Success ratio**: 42.82%<br>**Coefficients from all data(W)**:[[-0.11007982  1.27822786  0.4327677   4.70594637 -4.5398994  -1.15414932
   1.15746344]]

### Αποτελέσματα ridge regression με grid search για αρχικά δεδομένα :
**Cross-val explained variance**: 0.52<br>**Test set explained variance**: 0.53<br>**Success ratio**: 43.54%<br>**Coefficients from all data(W)**:[[  2.08464113   8.86057672   8.37715794   8.03320605 -18.65820307
   -7.89447003   9.84185308]]

In [6]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_predict

#Δημιουργία πολυωνύμου 2ου βαθμού
polyModel = PolynomialFeatures(degree = 2)
X_train_poly = polyModel.fit_transform(X_train)

X_test_poly = polyModel.fit_transform(X_test)

#Ridge regression στο πολυώνυμο
regr  = linear_model.Ridge(alpha=0.001)
y_train_pred = cross_val_predict(regr, X_train_poly, y_train, cv=10)
regr.fit(X_train_poly, y_train)
y_test_pred = regr.predict(X_test_poly)



display(Markdown(
    "### Αποτελέσματα ridge regression σε πολυωνυμο 2ου βαθμου"+ ":\n" +
    "**Cross-val explained variance**: {:.2f}".format(explained_variance_score(y_train, y_train_pred)) + "<br>" +
    "**Test set explained variance**: {:.2f}".format(explained_variance_score(y_test, y_test_pred))+"<br>"+
    "**Success ratio**: {:.2f}".format(success_rate(y_test_pred,y_test,1))+"%"
))

#Δημιουργία πολυωνύμου 2ου βαθμού για κανονικοποιημένα δεδομένα
polyModel = PolynomialFeatures(degree = 2)
X_train_poly_scaled = polyModel.fit_transform(X_train_scaled)
X_test_poly_scaled = polyModel.fit_transform(X_test_scaled)

#Ridge regression στο πολυώνυμο
regr  = linear_model.Ridge(alpha=0.001)
y_train_pred_scaled = cross_val_predict(regr, X_train_poly_scaled, y_train, cv=10)
regr.fit(X_train_poly_scaled, y_train)
y_test_pred_scaled = regr.predict(X_test_poly_scaled)


display(Markdown(
    "### Αποτελέσματα ridge regression σε πολυωνυμο 2ου βαθμου με κανονικοποιημένα δεδομένα"+ ":\n" +
    "**Cross-val explained variance**: {:.2f}".format(explained_variance_score(y_train, y_train_pred_scaled)) + "<br>" +
    "**Test set explained variance**: {:.2f}".format(explained_variance_score(y_test, y_test_pred_scaled))+"<br>"+
    "**Success ratio**: {:.2f}".format(success_rate(y_test_pred_scaled,y_test,1))+"%"
))

### Αποτελέσματα ridge regression σε πολυωνυμο 2ου βαθμου:
**Cross-val explained variance**: 0.45<br>**Test set explained variance**: 0.57<br>**Success ratio**: 46.89%

### Αποτελέσματα ridge regression σε πολυωνυμο 2ου βαθμου με κανονικοποιημένα δεδομένα:
**Cross-val explained variance**: 0.38<br>**Test set explained variance**: 0.57<br>**Success ratio**: 46.41%

In [7]:
from sklearn.neighbors import KNeighborsRegressor
regr = KNeighborsRegressor()
params = { 
           'n_neighbors': range(1,10),
            'weights': ['uniform', 'distance']
         }
grid = GridSearchCV(estimator=regr, cv = 10, param_grid=params) 
grid.fit(X_train, y_train) 

y_test_pred = grid.predict(X_test)

display(Markdown(
    "**k-Nearest Neighbours Regression**: <br>" +
    "**Best k**: {:.3f}".format(grid.best_params_['n_neighbors'])  + "<br>" +
    "**Best weighting strategy**: " + grid.best_params_['weights']  + "<br>" +
    "**Test set explained variance**: {:.2f}".format(explained_variance_score(y_test, y_test_pred))+ "<br>"+
    "**Success ratio**: {:.2f}".format(success_rate(y_test_pred,y_test,1))+"%"
))

**k-Nearest Neighbours Regression**: <br>**Best k**: 9.000<br>**Best weighting strategy**: distance<br>**Test set explained variance**: 0.56<br>**Success ratio**: 46.65%

**Η κανονικοποίηση όπως φαίνεται στα αποτελέσματα δεν βοήθησε τα μοντέλα** <br>
**Η σχέση μεταξύ του μοντέλου και του στόχου φαίνεται να είναι 2ου βαθμού ή μεγαλύτερη αφού τα καλύτερα αποτελέσματα τα πήραμε με πολυωνυμική παλινδρόμηση**